In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import sys, os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import tensorflow as tf
import tensorflow
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
print(tf.__version__)

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import numpy as np
# importing library for plotting 
from matplotlib import pyplot as plt 
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=90,
                                  horizontal_flip=True,
                                  vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=90,
                                  horizontal_flip=True,
                                  vertical_flip=True)
batch_size = 8

train_generator = train_datagen.flow_from_directory('train',
                                                target_size=(224,224),
#                                               color_mode="rgb",
                                                batch_size=batch_size,
                                                class_mode='categorical',
                                                shuffle=True)

val_generator = test_datagen.flow_from_directory('val',
                                            target_size=(224,224),
#                                           color_mode="rgb",
                                            batch_size=batch_size,
                                            class_mode='categorical',
                                            shuffle=True)

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

HEIGHT = 224
WIDTH = 224

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

In [ ]:
base_model.summary()

In [ ]:
nTrain = 6992
nVal = 1833

train_features = np.zeros(shape=(nTrain, 7, 7, 2048))
train_labels = np.zeros(shape=(nTrain,103)) # mention the no.of classes as label here

In [ ]:
val_features = np.zeros(shape=(nVal, 7, 7, 2048))
val_labels = np.zeros(shape=(nVal,103))

In [ ]:
#pass the image through the network which gives us a 7 x 7 x 512 dimensional Tensor
i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = base_model.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTrain:
        break
         
# train_features = np.reshape(train_features, (nTrain, 7 * 7 * 512))

In [ ]:
i = 0
for inputs_batch, labels_batch in val_generator:
    features_batch = base_model.predict(inputs_batch)
    val_features[i * batch_size : (i + 1) * batch_size] = features_batch
    val_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nVal:
        break
         
# val_features = np.reshape(val_features, (nVal, 7 * 7 * 512))

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, input_shape=(7,7,2048), kernel_size=(2,2), strides=(1,1)))
# model.add(Activation('relu'))
model.add(layers.LeakyReLU(alpha=0.05))

# model.add(Conv2D(filters=64, kernel_size=(2,2), strides=(1,1)))
# # model.add(Activation('relu'))
# model.add(layers.LeakyReLU(alpha=0.05))

# model.add(MaxPooling2D(pool_size=(2,2), strides=(1,1)))

model.add(Flatten())

model.add(Dense(256))
# model.add(Activation('relu'))
model.add(layers.LeakyReLU(alpha=0.05))

# model.add(Dense(512))
# # model.add(Activation('relu'))
# model.add(layers.LeakyReLU(alpha=0.05))

model.add(Dropout(rate = 0.5))

model.add(Dense(103))
model.add(Activation('softmax'))

model.summary()

In [ ]:
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_features,
                    train_labels,
                    epochs=20,
                    batch_size=batch_size,
                    validation_data=(val_features,val_labels))

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy on Resnet 50 Model')
plt.ylabel('Accuracy in percentage')
plt.xlabel('Epoch')
plt.legend(['train', 'validate'], loc='upper left')
# plt.savefig('TrainAcc_socrates_resnet2.png')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss on Resnet 50 Model')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validate'], loc='upper left')
# plt.savefig('Loss_socrates_resnet2.png')
plt.show()

In [ ]:
model.evaluate(test_features1,test_labels1)

In [ ]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size=(224,224),
                                            batch_size=8,
                                            class_mode='categorical')

In [ ]:
nTest = 994
# test_features = np.zeros(shape=(nTest, 7, 7, 2048))
# test_labels = np.zeros(shape=(nTest,103)) # mention the no.of classes as label here

In [ ]:
test_features1 = []
test_labels1 = []

In [ ]:
i = 0
for inputs_batch, labels_batch in test_set:
    features_batch = base_model.predict(inputs_batch)
    
    if len(features_batch) == 8:
        for k in range(8):
            test_features1.append(features_batch[k])
            test_labels1.append(labels_batch[k])
    
    i += 1
    if i * batch_size >= nTest:
        break

In [ ]:
test_features1 = np.asarray(test_features1)
test_labels1 = np.asarray(test_labels1)